## 9. 获取Wellington房产数据并进行预测

In [ ]:
print("🔄 获取Wellington房产数据进行预测...")
wellington_properties = get_wellington_properties(supabase_client)

if wellington_properties is not None and len(wellington_properties) > 0:
    print(f"✅ 获取到 {len(wellington_properties)} 条Wellington房产数据")
    
    # 显示数据样本
    print("\n📊 Wellington房产数据样本:")
    display(wellington_properties.head())
    
    # 进行预测
    prediction_results = predict_wellington_properties(
        wellington_properties, model, scaler, feature_names
    )
    
    if prediction_results is not None:
        # 显示预测结果
        print(f"\n📊 预测结果统计:")
        print(f"  总预测数量: {len(prediction_results)}")
        print(f"  平均置信度: {prediction_results['confidence_score'].mean():.4f}")
        print(f"  预测为可能出售: {len(prediction_results[prediction_results['predicted_status'] == 'likely to sell'])}")
        print(f"  预测为不太可能出售: {len(prediction_results[prediction_results['predicted_status'] == 'unlikely to sell'])}")
        
        # 显示高置信度预测结果
        high_confidence = prediction_results[prediction_results['confidence_score'] >= 0.8]
        print(f"\n🎯 高置信度(≥0.8)预测结果: {len(high_confidence)} 条")
        
        if len(high_confidence) > 0:
            print("\n🏠 高置信度Wellington房产预测:")
            for _, row in high_confidence.head(5).iterrows():
                status_emoji = "🟢" if row['predicted_status'] == "likely to sell" else "🔴"
                print(f"{status_emoji} {row['address']}")
                print(f"    地区: {row['suburb']} | {row['bedrooms']}房{row['bathrooms']}卫 | {row['year_built']}年建")
                print(f"    价格: ${row['price']:,} | 预测: {row['predicted_status']} | 置信度: {row['confidence_score']:.3f}")
                print()
                
        # 保存预测结果到CSV
        prediction_results.to_csv('wellington_property_predictions.csv', index=False)
        print(f"💾 预测结果已保存到 wellington_property_predictions.csv")
        
        # 保存到数据库
        if supabase_client:
            inserted_count = save_predictions_to_property_status(supabase_client, prediction_results)
            if inserted_count > 0:
                print(f"✅ 成功保存 {inserted_count} 条预测结果到property_status表")
else:
    print("❌ 无法获取Wellington房产数据")

## 10. 保存模型

In [ ]:
# 保存模型和相关数据
model_filename = 'wellington_property_model.joblib'
joblib.dump(model, model_filename)
print(f"✅ 模型已保存为 {model_filename}")

feature_columns_filename = 'feature_columns.joblib'
joblib.dump(feature_names, feature_columns_filename)
print(f"✅ 特征列已保存为 {feature_columns_filename}")

scaler_filename = 'scaler.joblib'
joblib.dump(scaler, scaler_filename)
print(f"✅ 标准化器已保存为 {scaler_filename}")

# 如果有预测结果，保存到CSV
if 'prediction_results' in locals() and prediction_results is not None:
    prediction_results.to_csv('wellington_property_predictions.csv', index=False)
    print(f"✅ 预测结果已保存到 wellington_property_predictions.csv")

## 11. 下载模型文件

In [ ]:
from google.colab import files

try:
    files.download(model_filename)
    files.download(feature_columns_filename)
    files.download(scaler_filename)
    if 'prediction_results' in locals() and prediction_results is not None:
        files.download('wellington_property_predictions.csv')
    print("✅ 模型文件下载成功!")
except:
    print("注意: 下载功能只在Google Colab环境中有效")

## 12. 结果总结

In [ ]:
print("🎉 Wellington房产预测完成!")
print("=" * 60)

if 'test_accuracy' in locals():
    print(f"✅ 模型准确率: {test_accuracy:.4f}")
    
if 'prediction_results' in locals() and prediction_results is not None:
    print(f"✅ 总预测数量: {len(prediction_results)}")
    high_conf_count = len(prediction_results[prediction_results['confidence_score'] >= 0.8])
    print(f"✅ 高置信度(≥0.8): {high_conf_count}")
    print(f"✅ 平均置信度: {prediction_results['confidence_score'].mean():.4f}")
    
    likely_to_sell = len(prediction_results[prediction_results['predicted_status'] == 'likely to sell'])
    print(f"✅ 预测可能出售: {likely_to_sell} 个房产")

print("\n📋 关键发现:")
print("  • 基于real_estate表的真实销售数据训练模型")
print("  • 对properties表中的Wellington房产进行预测")
print("  • 识别出最有可能近期出售的房产")
print("  • 预测结果已保存到property_status表")

print("\n🚀 使用建议:")
print("  1. 重点关注置信度≥0.8的预测结果")
print("  2. 优先推荐'likely to sell'状态的房产")
print("  3. 结合房产特征（年龄、价格、地区）进行分析")
print("  4. 定期更新模型以提高预测准确性")
print("  5. 查看property_status表获取完整预测数据")

print("\n" + "="*60)
if 'supabase_client' in locals() and supabase_client:
    print("🎯 Wellington房产预测完成，结果已保存到数据库，可以在应用中展示！")
else:
    print("⚠️ 请配置数据库连接以启用自动保存功能")
print("="*60)